In [1]:
from datetime import datetime
from pprint import pprint
from itertools import *

import textblob
from textblob import TextBlob

import pandas as pd
pd.set_option('display.max_columns', None)
# import swifter

import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
# EntityTypes = ["PERSON", "NORP", "ORG", "FAC", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]

In [3]:
df = pd.read_csv(r"D:\Work\Coding\Data\some-news.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("Unnamed: 0.1", axis=1)
df['article'] = df['article'].astype(str)

In [4]:
def getEnts(i):
    doc = nlp(i)
    entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}
    
    return entities

def addEnts(dataframe, columnName):

    extracted = []

    for i, row in dataframe.iterrows():
        ents = getEnts(row[columnName])
        extracted.append(ents)

    dataframe["ExtractedEnts"] = extracted
    
    return dataframe

def splitEnts(dataframe):
    df_temp = pd.json_normalize(dataframe['ExtractedEnts'])
    dataframe = pd.concat([dataframe, df_temp], axis=1)
    
    return dataframe

In [5]:
starttime = datetime.now()
print(starttime)

df = addEnts(df, 'article')

print(f"{datetime.now() - starttime}")

2021-05-21 00:22:12.300410
0:01:26.180538


In [6]:
starttime = datetime.now()
print(starttime)

df = splitEnts(df)

print(f"{datetime.now() - starttime}")

2021-05-21 00:23:38.490948
0:00:00.041015


In [11]:
starttime = datetime.now()
print(starttime)

df['sentiment'] = df['article'].apply(lambda article: TextBlob(article).sentiment)
print(f"Added Sentiment Analysis in {datetime.now() - starttime} seconds")

2021-05-21 00:25:13.026809
Added Sentiment Analysis in 0:00:03.024162 seconds


In [17]:
starttime = datetime.now()
print(starttime)

df[['Polarity','Subjectivity']] = pd.DataFrame(df['sentiment'].to_list(), columns=['Polarity','Subjectivity'])

print(f"Split columns in {datetime.now() - starttime} seconds")

2021-05-21 00:28:09.369593
Split columns in 0:00:00.001998 seconds


In [18]:
df

,date,year,month,day,author,title,article,url,section,publication,ExtractedEnts,CARDINAL,DATE,GPE,LOC,ORG,PERSON,MONEY,ORDINAL,PRODUCT,NORP,PERCENT,QUANTITY,TIME,WORK_OF_ART,EVENT,FAC,LANGUAGE,LAW,sentiment,Polarity,Subjectivity
0,09-12-16 18:31,2016,12,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox,"{'CARDINAL': ['30', 'one'], 'DATE': ['the next...","[30, one]","[the next year, the mid-2000s, 30s, the past t...","[Voeten, Washington]","[Polyarchy, New America]","[the New York Times, Journal of Democracy, Mou...","[Amanda Taub's, Foa, Roberto Stefan Foa, Mounk...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.059539423680645055, 0.5074067832083098)",0.059539,0.507407
1,07-10-16 21:26,2016,10,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider,"{'CARDINAL': ['32'], 'DATE': ['last year's', '...",[32],"[last year's, this year, a year ago, Friday, f...","[Indianapolis, Denver]",NaN,"[DVOA, Patriots, NFL]","[Peyton Manning, Luck, Keefer, Andrew Luck, Ma...","[$89 million, $18.4 million, $17.5 million, $1...",[30th],"[Colts GM Ryan Grigson, Colts, Broncos]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.06238095238095237, 0.6090476190476191)",0.062381,0.609048
2,26-01-18 00:00,2018,1,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters,"{'CARDINAL': ['four'], 'DATE': ['Friday', 'las...",[four],"[Friday, last June, Thursday]","[U.S., DAVOS, Davos, Switzerland]",NaN,"[the White House, The New York Times, Trump, R...","[Donald Trump, Steve Holland, Robert Mueller]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(-0.14525613275613275, 0.5476911976911977)",-0.145256,0.547691
3,27-06-19 00:00,2019,6,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters,"{'DATE': ['Thursday', '2014', '2017', 'May', '...",NaN,"[Thursday, 2014, 2017, May, 2007-2012, 2012, T...","[PARIS, France, Elysee]",NaN,"[Le Point, Macron, Reuters]","[Nicolas Sarkozy, Les Republicains, Bling-Blin...",NaN,[first],NaN,"[European, French]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.03248283702829158, 0.31876995058813246)",0.032483,0.318770
4,27-01-16 00:00,2016,1,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ,"{'DATE': ['Wednesday', 'Sunday'], 'GPE': ['Par...",NaN,"[Wednesday, Sunday]","[Paris, Paris Hilton, Switzerland]",NaN,NaN,"[Monty Brinson, Monty, Kim Richards]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.13425925925925927, 0.5620370370370371)",0.134259,0.562037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,20-07-18 00:00,2018,7,20,NaN,Christian Bale Shows Off Extreme Weight Loss A...,Christian Bale's ditched any vestige of Dick C...,https://www.tmz.com/2018/07/20/christian-bale-...,NaN,TMZ,"{'DATE': ['7 months ago', 'Friday', 'months'],...",NaN,"[7 months ago, Friday, months]",[Italy],NaN,"[CB, Machinist]","[Dick Cheney, Christian Bale's]",NaN,NaN,NaN,[Christian],NaN,"[roughly 100 lbs, about 228 pounds, almost 70 ...",[morning],[American Hustle],NaN,NaN,NaN,NaN,"(0.05863095238095238, 0.4419642857142857)",0.058631,0.441964
991,07-03-16 16:00,2016,3,7,Tim Shiel,THUMP Australia's Monthly Oceanic Report Will ...,The Oceanic Report is our new column bringing ...,https://www.vice.com/en_us/article/wnyx4w/thum...,Noisey,Vice,"{'CARDINAL': ['ten'], 'DATE': ['2016', 'This m...",[ten],"[2016, This month, Hiatus]","[Sydney, Perth, Kaiyote, Rarotonga, New Zealan...","[Europe, Cook Islan

In [24]:
df = df.drop(columns=['ExtractedEnts','sentiment'])

In [19]:
df.sort_values("Subjectivity")

,date,year,month,day,author,title,article,url,section,publication,ExtractedEnts,CARDINAL,DATE,GPE,LOC,ORG,PERSON,MONEY,ORDINAL,PRODUCT,NORP,PERCENT,QUANTITY,TIME,WORK_OF_ART,EVENT,FAC,LANGUAGE,LAW,sentiment,Polarity,Subjectivity
813,09-09-16 00:00,2016,9,9,Nayland Blake,What Can't Be Seen,Advertise on Hyperallergic with Nectar Ads,https://hyperallergic.com/321891/what-cant-be-...,NaN,Hyperallergic,{'ORG': ['Nectar']},NaN,NaN,NaN,NaN,[Nectar],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.0, 0.0)",0.000000,0.000000
893,05-07-19 00:00,2019,7,5,NaN,Russia's VTB increases stake in housebuilder P...,"MOSCOW, July 5 (Reuters) - VTB has increased i...",https://www.reuters.com/article/russia-vtb-pik...,Financials,Reuters,"{'DATE': ['July 5', 'Friday'], 'GPE': ['Russia...",NaN,"[July 5, Friday]","[Russia, MOSCOW]",NaN,"[PIK Group, VTB, Reuters]","[Maxim Rodionov, Jason Neely]",NaN,[second],NaN,NaN,"[7.57%, 23.05%]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.0, 0.0)",0.000000,0.000000
277,29-03-19 00:00,2019,3,29,NaN,Voting for UK PM May's Brexit Withdrawal Agree...,EDINBURGH (Reuters) - Voting for Prime Ministe...,https://www.reuters.com/article/us-britain-eu-...,World News,Reuters,"{'DATE': ['May', 'Friday'], 'GPE': ['Britain']...",NaN,"[May, Friday]",[Britain],NaN,"[EU, LBC, the European Union, EDINBURGH, Reuters]","[pro-Brexit, Guy Faulconbridge, Mark Francois,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.0, 0.0)",0.000000,0.000000
632,03-07-19 00:00,2019,7,3,NaN,U.S. sells 2-month bills at high rate 2.155%,July 3 (Reuters) - For details of the U.S. Tre...,https://www.reuters.com/article/usa-debt-bills...,Bonds News,Reuters,"{'DATE': ['2-month', '8-week', 'July 3'], 'ORG...",NaN,"[2-month, 8-week, July 3]",NaN,NaN,"[Reuters, the U.S. Treasury’s]",[Richard Leong],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.0, 0.0)",0.000000,0.000000
187,29-07-16 00:00,2016,7,29,Nayland Blake,Sun Ra Had It Right,Advertise on Hyperallergic with Nectar Ads,https://hyperallergic.com/314667/sun-ra-had-it...,NaN,Hyperallergic,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.0, 0.0)",0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,18-06-19 00:00,2019,6,18,NaN,BRIEF-Edisun Power Europe Decides On Capital I...,June 18 (Reuters) - EDISUN POWER EUROPE AG: *...,https://www.reuters.com/article/brief-edisun-p...,Switzerland Market Report,Reuters,"{'CARDINAL': ['2,423,460.00'], 'DATE': ['DECEM...","[2,423,460.00]","[DECEMBER 31, 2019, June 18]",NaN,NaN,"[Gdynia Newsroom, THE SIX SWISS EXCHANGE IS, E...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.09727272727272726, 0.7509090909090909)",0.097273,0.750909
524,08-05-16 00:00,2016,5,8,NaN,Miranda Kerr and Snapchat's Evan Spiegel Shack...,Miranda Kerr and Snapchat founder Evan Spiegel...,https://www.tmz.com/2016/05/08/miranda-kerr-ev...,NaN,TMZ,"{'CARDINAL': ['6', '4', 'one'], 'GPE': ['Brent...","[6, 4, one]",NaN,"[Brentwood, California]",NaN,NaN,"[Miranda Kerr, O.J., Evan Spiegel, Snapchat, G...","[$12.5 mil, $12 mil]",NaN,NaN,NaN,NaN,"[almost an acre, 7,100 square foot]",NaN,NaN,NaN,NaN,NaN,NaN,"(0.5183333333333333, 0.7783333333333333)",0.518333,0.778333
153,14-06-19 00:00,2019,6,14,NaN,BUZZ-Scisys: Jumps after CGI offers to buy co ...,** IT services developer Scisys’ up 21.1% at 2...,https://www.reuters.com/article/buzz-scisys-ju...,Hot Stocks,Reuters,"{'CARDINAL': ['254.15p'], 'DATE': ['Thursday']...",[254.15p],[Thursday],[Bengaluru],NaN,"[Inc, 250p, SSY , SSY]","[Shariq Khan, Scisys]","[0.7892 pounds, 1, 78.9 mln pounds, $99.97 mln]",NaN,[204p],[Canadian],"[21.1%, 24.6%, ~37.4%]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(0.25, 0.8125)",0.250000,0.812500
98,20-04-18 00:00,2018,4,20,NaN,TMZ Live Cardi B & Bernie Sanders Join Forces ...,HERE'S THE RUNDOWNTristan Thompson Leaves Khlo...,https://www.tmz.com/2018/04/20/tmz-live/,NaN,TMZ,"{'PERSON': ['Khloe', 'ClevelandPr

In [27]:
import os

os.getcwd()

'D:\\Work\\Coding\\NLP-Tools\\Information Extraction'

In [28]:
df.to_csv("Some News (Tagged).csv")